In [6]:
# Cell 1 — Imports & Constants
import os, glob, numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from tqdm.notebook import tqdm
import pandas as pd

# Paths
DEN_DIR = '/home/jovyan/Features/denoised'
EMB_DIR = '/home/jovyan/Features/embeddings'

# Spectrogram params (must match your preprocessing)
PANNS_SR     = 32000
N_FFT        = 2048
HOP_LENGTH   = 512
N_MELS       = 128
ALPHA        = 0.5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)


meta = pd.read_csv('/home/jovyan/Data/birdclef-2025/train.csv')
label2idx = {lab:i for i, lab in enumerate(sorted(meta['primary_label'].unique()))}
num_classes = len(label2idx)


Using device: cuda


In [7]:
# Cell 2 — Helpers: MEL & Augmentation
import librosa
from sklearn.preprocessing import StandardScaler

def calculate_mel_spectrogram(wave_np, sr=PANNS_SR,
                              n_fft=N_FFT, hop_length=HOP_LENGTH,
                              n_mels=N_MELS):
    S = librosa.feature.melspectrogram(
        y=wave_np, sr=sr,
        n_fft=n_fft, hop_length=hop_length,
        n_mels=n_mels
    )
    return librosa.power_to_db(S, ref=np.max)

def create_augmented_mel(log_mel, emb, n_mels=N_MELS, alpha=ALPHA):
    dim, T = emb.shape[0], log_mel.shape[1]
    if dim == n_mels:
        proj = emb
    elif dim > n_mels:
        factor = dim // n_mels
        proj = emb[:n_mels*factor].reshape(n_mels, factor).mean(axis=1) \
               if dim % n_mels == 0 else emb[:n_mels]
    else:
        proj = np.pad(emb, (0, n_mels - dim))
    tiled = np.tile(proj[:, None], (1, T))
    normed = StandardScaler().fit_transform(tiled.T).T
    return log_mel + alpha * normed


In [8]:
# Cell 3 — Dataset for Augmented MEL as Image
class AugmentedMelImageDataset(Dataset):
    def __init__(self, den_dir, emb_dir, transform=None):
        self.den_paths = sorted(glob.glob(f"{den_dir}/**/*.npz", recursive=True))
        self.emb_dir   = emb_dir
        self.transform = transform

    def __len__(self):
        return len(self.den_paths)

    def __getitem__(self, idx):
        dpath = self.den_paths[idx]
        d = np.load(dpath)
        wave = d['waveform']           # [CHUNK_SAMPLES]
        lbl  = int(d['label'])

        # load embedding
        rel   = os.path.relpath(dpath, DEN_DIR)
        emb_p = os.path.join(self.emb_dir, rel.replace('.npz','_emb.npz'))
        emb   = np.load(emb_p)['embedding']

        # compute spectrograms
        mel = calculate_mel_spectrogram(wave)
        aug = create_augmented_mel(mel, emb)

        # to “image”
        img = aug.astype(np.float32)   # H x W
        img = np.expand_dims(img, 0)   # 1 x H x W

        if self.transform:
            img = self.transform(torch.from_numpy(img))

        return img, lbl


In [9]:
# Cell 4 — Transforms & DataLoaders
# Resize to 224×224 and duplicate channel→3
train_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.Lambda(lambda x: x.repeat(3,1,1)),      # 1→3 channels
    transforms.Normalize([0.5]*3, [0.5]*3)
])
val_tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.Lambda(lambda x: x.repeat(3,1,1)),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataset = AugmentedMelImageDataset(DEN_DIR, EMB_DIR, transform=None)
# simple 80/20 split
n = len(dataset)
n_train = int(0.8*n)
train_ds, val_ds = torch.utils.data.random_split(dataset, [n_train, n-n_train])

train_ds.dataset.transform = train_tf
val_ds.dataset.transform   = val_tf

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,
                          num_workers=4, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=32, shuffle=False,
                          num_workers=4, pin_memory=True)
print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")


Train batches: 281, Val batches: 71


In [11]:
# Cell 5 — Model, Loss, Optimizer
# ResNet-50 from scratch
model = models.resnet50(weights=None, num_classes=num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# count params
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total trainable params:", total_params)


Total trainable params: 23930126


In [13]:
# Cell 6 — Train & Validate Loop (2 epochs example)
num_epochs = 1
best_acc = 0.0
for epoch in range(1, num_epochs+1):
    # Train
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch} Train"):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()*imgs.size(0)
        preds = outputs.argmax(1)
        correct += (preds==labels).sum().item()
        total += imgs.size(0)

    train_loss = running_loss/total
    train_acc  = correct/total

    # Validate
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch} Val"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()*imgs.size(0)
            preds = outputs.argmax(1)
            val_correct += (preds==labels).sum().item()
            val_total   += imgs.size(0)

    val_loss /= val_total
    val_acc   = val_correct/val_total

    print(f"Epoch {epoch}: Train loss {train_loss:.4f}, acc {train_acc:.4f} | "
          f"Val loss {val_loss:.4f}, acc {val_acc:.4f}")

    # Save best
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_resnet50.pth')
        print(f"➡️  New best saved (acc {best_acc:.4f})")

print("Finished. Best Val Acc:", best_acc)


Epoch 1 Train:   0%|          | 0/281 [00:00<?, ?it/s]

Epoch 1 Val:   0%|          | 0/71 [00:00<?, ?it/s]

Epoch 1: Train loss 4.8893, acc 0.0339 | Val loss 4.6996, acc 0.0405
➡️  New best saved (acc 0.0405)
Finished. Best Val Acc: 0.04048042704626335
